## Installing External Libraries

In [1]:
!pip install efficientnet_pytorch --quiet

## Libraries

In [2]:
################ Importing Libraries ################

import os
import torch
import random
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import albumentations as A
import torch.cuda.amp as amp
from torchinfo import summary
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from timeit import default_timer as timer
from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
!mkdir Models

pd.set_option('expand_frame_repr', False)
print(torch.__version__, torchvision.__version__)

1.13.0 0.14.0


## Configurations

In [3]:
## Configuration Settings ##

class Config:
        
    EPOCHS = 20
    IMG_SIZE = 512
    ES_PATIENCE = 2
    WEIGHT_DECAY = 0.001
    VAL_BATCH_SIZE = 32 * 2
    RANDOM_STATE = 1994
    LEARNING_RATE = 5e-5
    TRAIN_BATCH_SIZE = 32
    MEAN = (0.485, 0.456, 0.406)
    STD = (0.229, 0.224, 0.225)
    TRAIN_COLS = ["image_name", "patient_id", "sex", "age_approx", "anatom_site_general_challenge",
                 "target", "tfrecord"]
    TEST_COLS = ["image_name", "patient_id", "sex", "age_approx", "anatom_site_general_challenge"]
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    ################ Setting paths to data input ################
    
    data_2020 = "/kaggle/input/jpeg-melanoma-512x512/"
    train_folder_2020 = data_2020 + "train/"
    test_folder_2020 = data_2020 + "test/"
    test_csv_path_2020 = data_2020 + "test.csv"
    train_csv_path_2020 = data_2020 + "train.csv"
    submission_csv_path = data_2020 + "sample_submission.csv"

## Utilities

In [4]:
## Helper Utilities

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience.
       Directly borrowed from https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py"""
    def __init__(self, path, patience=7, verbose=False, delta=0, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(obj=model.state_dict(), f=self.path)
        self.val_loss_min = val_loss
        
def plot_loss_curves(results):
    """
    Function to plot training & validation loss curves & validation AUC
    """
    loss = results['train_loss']
    valid_loss = results['valid_loss']
    # Get the accuracy values of the results dictionary (training and test)
    valid_auc = results['valid_auc']
    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))
    # Setup a plot 
    plt.figure(figsize=(15, 7))
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, valid_loss, label='valid_loss')
    plt.title('Loss'); plt.xlabel('Epochs');plt.legend()
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, valid_auc, label='valid_auc')
    plt.title('AUC Score'); plt.xlabel('Epochs'); plt.legend();
    
class RareLabelCategoryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, variables, tol=0.05):
        if not isinstance(variables, list):
            raise ValueError('Variables should be a list')
        self.tol = tol
        self.variables = variables
    def fit(self, X, y=None):
        self.encoder_dict_ = {}
        for var in self.variables:
            t = pd.Series(X[var]).value_counts(normalize=True)
            self.encoder_dict_[var] = list(t[t >= self.tol].index)
        return self
    def transform(self, X):
        X = X.copy()
        for var in self.variables:
            X[var] = np.where(
                X[var].isin(self.encoder_dict_[var]),
                                        X[var], "Other")
        return X
    
class OutlierTreatment(BaseEstimator, TransformerMixin):
    def __init__(self, variable, upper_quantile=None, lower_quantile=None):
        if not isinstance(variable, str):
            raise ValueError("Variable should be a string type.")
        self.upper_quantile = upper_quantile
        self.variable = variable
        self.lower_quantile = lower_quantile
    def fit(self, X, y=None):
        self.upper_quantile = X[self.variable].quantile(self.upper_quantile)
        self.lower_quantile = X[self.variable].quantile(self.lower_quantile)
        return self
    def transform(self, X):
        X = X.copy()
        X[self.variable] = np.where(
                    X[self.variable] > self.upper_quantile, self.upper_quantile,
                            np.where(X[self.variable] < self.lower_quantile, self.lower_quantile, X[self.variable]))
        return X

## Dataset

In [5]:
## Creating Dataset classes to load the images

ImageFile.LOAD_TRUNCATED_IMAGES = True

class DatasetRetriever(nn.Module):
    def __init__(self, df, tabular_features=None, use_tabular_features=False,
                augmentations=None, is_test=False):
        self.df = df
        self.tabular_features = tabular_features
        self.use_tabular_features = use_tabular_features
        self.augmentations = augmentations
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_path = self.df['image_path'].iloc[index]
        image = Image.open(image_path)
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        image = torch.tensor(image, dtype=torch.float)
        if self.use_tabular_features:
            if len(self.tabular_features) > 0 and self.is_test == False:
                tabular_features = np.array(self.df.iloc[index][self.tabular_features].values, 
                                            dtype=np.float32)
                targets = self.df.target[index]
                return {"image": image, "tabular_features": tabular_features, "targets": torch.tensor(targets, 
                                                                                                      dtype=torch.long)}
            elif len(self.tabular_features) > 0 and self.is_test == True:
                tabular_features = np.array(self.df.iloc[index][self.tabular_features].values,
                                            dtype=np.float32)
                return {"image": image, "tabular_features": tabular_features}
        else:
            if self.is_test == False:
                targets = self.df.target[index]
                return {"image" : image, "targets": torch.tensor(targets, dtype=torch.long)}
            elif self.is_test == True:
                return {"image": image}

## Training & Evaluation Loops

In [6]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device, scaler, use_tabular_features=True):
    train_loss = 0
    model.train()
    for batch, data in enumerate(dataloader):
        optimizer.zero_grad()
        if use_tabular_features:
            if batch == 0:
                print("Using tabular features")
            data["image"], data["tabular_features"], data['targets'] = data["image"].to(device, dtype=torch.float), \
                    data["tabular_features"].to(device, dtype=torch.float), data['targets'].to(device, dtype=torch.float)
            with amp.autocast():
                y_logits = model(data['image'], data['tabular_features']).squeeze(dim=0)
                loss = loss_fn(y_logits, data["targets"].view(-1,1))
        else:
            if batch == 0:
                print("Not using tabular features")
            data["image"], data['targets'] = data["image"].to(device, dtype=torch.float), data['targets'].to(device, 
                                                                                        dtype=torch.float)
            with amp.autocast():
                y_logits = model(data['image']).squeeze(dim=0)
                loss = loss_fn(y_logits, data["targets"].view(-1,1))
        train_loss += loss.item()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()        
    train_loss = train_loss / len(dataloader)
    return train_loss

def validate_one_epoch(model, dataloader, loss_fn, device, use_tabular_features=True):
    valid_loss, final_predictions = 0, []
    model.eval()
    with torch.inference_mode():
        for batch, data in enumerate(dataloader):
            if use_tabular_features:
                if batch == 0:
                    print("Using tabular features")
                data["image"], data["tabular_features"], data['targets'] = data["image"].to(device, dtype=torch.float), \
                    data["tabular_features"].to(device, dtype=torch.float), data['targets'].to(device, dtype=torch.float)
                y_logits = model(data['image'], data['tabular_features']).squeeze(dim=0)
            else:
                if batch == 0:
                    print("Not using tabular features")
                data["image"], data['targets'] = data["image"].to(device, dtype=torch.float), data['targets'].to(device,
                                                                                                    dtype=torch.float)    
                y_logits = model(data['image']).squeeze(dim=0)
            loss = loss_fn(y_logits, data["targets"].view(-1,1))
            valid_loss += loss.item()
            valid_probs = torch.sigmoid(y_logits).detach().cpu().numpy()
            final_predictions.extend(valid_probs)
    valid_loss = valid_loss / len(dataloader)
    return valid_loss, final_predictions

def train(model, train_dataloader, valid_dataloader, loss_fn, optimizer, scheduler,
          device, scaler, epochs, es_patience, model_save_path, validation_targets):
    results = {"train_loss": [], "valid_loss": [], "valid_auc": []}
    
    early_stopping = EarlyStopping(patience=es_patience, verbose=True, path=model_save_path)
    
    for epoch in tqdm(range(epochs)):
        train_loss = train_one_epoch(model=model, dataloader=train_dataloader,
                                    loss_fn=loss_fn, optimizer=optimizer,
                                    device=device, scaler=scaler, use_tabular_features=True)
        
        valid_loss, valid_predictions = validate_one_epoch(model=model, 
                                    dataloader=valid_dataloader, loss_fn=loss_fn, device=device,
                                                          use_tabular_features=True)
        
        valid_predictions = np.vstack(valid_predictions).ravel()
        
        valid_auc = roc_auc_score(y_score=valid_predictions, 
                                  y_true=validation_targets)
        scheduler.step(valid_auc)
        
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print(f"Early Stopping")
            break
            
        model.load_state_dict(torch.load(model_save_path))
        print(f"Epoch : {epoch+1} | "
              f"train_loss : {train_loss:.4f} | "
              f"valid_loss : {valid_loss:.4f} | "
              f"valid_auc : {valid_auc:.4f} ")
        results['train_loss'].append(train_loss)
        results['valid_loss'].append(valid_loss)
        results['valid_auc'].append(valid_auc)
    return results

## Augmentations

In [7]:
## Setting Training & Validation Augmentations

training_augmentations = A.Compose([A.CoarseDropout(p=0.6),
                                    A.RandomRotate90(p=0.6),
                                    A.Flip(p=0.4),
                                    A.OneOf([A.RandomBrightnessContrast(brightness_limit=0.2,
                                                                       contrast_limit=0.3),
                                            A.HueSaturationValue(hue_shift_limit=20,
                                                                sat_shift_limit=60,
                                                                val_shift_limit=50)], p=0.7),
                                    A.OneOf([A.GaussianBlur(),
                                            A.GaussNoise()], p=0.65),
                                    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.35, rotate_limit=45, p=0.5),
                                    A.OneOf([A.OpticalDistortion(p=0.3),
                                            A.GridDistortion(p=0.1),
                                            A.PiecewiseAffine(p=0.3)], p=0.7),
    A.Normalize(mean=Config.MEAN, std=Config.STD,
                max_pixel_value=255.0, always_apply=True)
])

validation_augmentations = A.Compose([A.Normalize(mean=Config.MEAN, std=Config.STD, 
                                        max_pixel_value=255.0,always_apply=True)
                                     ])
testing_augmentations = A.Compose([A.Normalize(mean=Config.MEAN, std=Config.STD, 
                                        max_pixel_value=255.0,always_apply=True)
                                     ])

## Model

In [8]:
class Model(nn.Module):
    def __init__(self, model_name='efficientnet-b5', pool_type=F.adaptive_avg_pool2d,
                num_tabular_features=0):
        super().__init__()
        self.pool_type = pool_type
        self.model_name = model_name
        self.backbone = EfficientNet.from_pretrained(model_name)
        in_features = getattr(self.backbone, "_fc").in_features
        if num_tabular_features>0:
            self.meta = nn.Sequential(
                nn.Linear(num_tabular_features, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.Linear(512, 128),
                nn.BatchNorm1d(128),
                nn.ReLU())
            in_features += 128
        self.output = nn.Linear(in_features, 1)
    
    def forward(self, image, tabular_features=None):
        features = self.pool_type(self.backbone.extract_features(image), 1)
        cnn_features = features.view(image.size(0),-1)
        if num_tabular_features>0:
            tabular_features = self.meta(tabular_features)
            all_features = torch.cat((cnn_features, tabular_features), dim=1)
            output = self.output(all_features)
            return output
        else:
            output = self.output(cnn_features)
            return output

## Reading the Data

In [9]:
## Looking at the Data

train_df = pd.read_csv(Config.train_csv_path_2020,
                       usecols=Config.TRAIN_COLS)
print(f"Number of Data points & Columns in Training dataset are - {train_df.shape}\n")

test_df = pd.read_csv(Config.test_csv_path_2020,
                       usecols=Config.TEST_COLS)
print(f"Number of Data points & Columns in 2020 Testing dataset are - {test_df.shape}\n")

print(f"Train Dataset Columns: {Config.TRAIN_COLS}\n")
print(f"Test Dataset Columns: {Config.TEST_COLS}\n")

print(f"Distribution of Target feature in training dataset are: \n{train_df['target'].value_counts(normalize=True) * 100}\n")
print("*"*70 + "\n")

print(f"Distribution of sex feature in training dataset are:\n{train_df['sex'].value_counts(normalize=True, dropna=False) * 100}\n")
print("*"*50 + "\n")
print(f"Distribution of sex feature in testing dataset are:\n{test_df['sex'].value_counts(normalize=True, dropna=False) * 100}\n")

print(f"Distribution of anatom_site_general_challenge feature in training dataset are:\n\n{train_df['anatom_site_general_challenge'].value_counts(normalize=True, dropna=False) * 100}\n")
print("*"*70 + "\n")
print(f"Distribution of anatom_site_general_challenge feature in testing dataset are:\n\n{test_df['anatom_site_general_challenge'].value_counts(normalize=True, dropna=False) * 100}")
print("*"*70 + "\n")

print(f"------------------- Missing values distribution in training dataset-------------------:\n{train_df.isnull().sum()}\n")
print(f"-------------------Missing values distribution in testing dataset-------------------:\n{test_df.isnull().sum()}\n")

print('\n\nViewing Training Dataset below\n')
print(train_df.head())
print("*"*100 + "\n")
print('\n\nViewing Testing Dataset below\n')
print(test_df.head())

Number of Data points & Columns in Training dataset are - (33126, 7)

Number of Data points & Columns in 2020 Testing dataset are - (10982, 5)

Train Dataset Columns: ['image_name', 'patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge', 'target', 'tfrecord']

Test Dataset Columns: ['image_name', 'patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge']

Distribution of Target feature in training dataset are: 
0    98.237034
1     1.762966
Name: target, dtype: float64

**********************************************************************

Distribution of sex feature in training dataset are:
male      51.560708
female    48.243072
NaN        0.196220
Name: sex, dtype: float64

**************************************************

Distribution of sex feature in testing dataset are:
male      56.956838
female    43.043162
Name: sex, dtype: float64

Distribution of anatom_site_general_challenge feature in training dataset are:

torso              50.851295
lower extrem

In [10]:
## Filling Missing values.
train_df['sex'] = train_df['sex'].fillna("missing")
test_df['sex'] = test_df['sex'].fillna("missing")
train_df['anatom_site_general_challenge'] = train_df['anatom_site_general_challenge'].fillna("missing")
test_df['anatom_site_general_challenge'] = test_df['anatom_site_general_challenge'].fillna("missing")
train_df['age_approx'] = train_df['age_approx'].fillna(-1)
test_df['age_approx'] = test_df['age_approx'].fillna(-1)

## Combining Rare categories that appear less than 5%.
rare_label = RareLabelCategoryEncoder(tol=0.05, variables=['anatom_site_general_challenge'])
rare_label.fit(train_df)
train_df = rare_label.transform(train_df)
test_df = rare_label.fit_transform(test_df)

## Capping outlier values in Age feature.
outlier_treat = OutlierTreatment(variable="age_approx", lower_quantile=0.01, upper_quantile=1.0)
outlier_treat.fit(train_df)
train_df = outlier_treat.transform(train_df)
test_df = outlier_treat.transform(test_df)

## Replacing characters in AnatomSiteGeneralChallenge feature
train_df['anatom_site_general_challenge'] = train_df['anatom_site_general_challenge'].str.replace("/","_")
test_df['anatom_site_general_challenge'] = test_df['anatom_site_general_challenge'].str.replace("/","_")
train_df['anatom_site_general_challenge'] = train_df['anatom_site_general_challenge'].str.replace(" ","_")
test_df['anatom_site_general_challenge'] = test_df['anatom_site_general_challenge'].str.replace(" ","_")

## Creating OneHotEncoded features on Sex & AnatomSiteGeneralChallene Feature.
sex_one_hot = OneHotEncoder(sparse=False, handle_unknown="ignore", drop="first")
sex_one_hot.fit(train_df[['sex']])
train_sex_features = pd.DataFrame(sex_one_hot.transform(train_df[['sex']]),
                                  columns=sex_one_hot.get_feature_names_out().tolist())
test_sex_features = pd.DataFrame(sex_one_hot.transform(test_df[['sex']]),
                                  columns=sex_one_hot.get_feature_names_out().tolist())

anatom_one_hot = OneHotEncoder(sparse=False, handle_unknown="ignore", drop="first")
anatom_one_hot.fit(train_df[['anatom_site_general_challenge']])
train_anatom_features = pd.DataFrame(anatom_one_hot.transform(train_df[['anatom_site_general_challenge']]), 
                                     columns=anatom_one_hot.get_feature_names_out().tolist())
test_anatom_features = pd.DataFrame(anatom_one_hot.transform(test_df[['anatom_site_general_challenge']]), 
                                    columns=anatom_one_hot.get_feature_names_out().tolist())

## Scaling Age feature
age_scaler = StandardScaler()
age_scaler.fit(train_df[['age_approx']])
train_scaled_age = pd.DataFrame(age_scaler.transform(train_df[['age_approx']]), columns=['scaled_age'])
test_scaled_age = pd.DataFrame(age_scaler.transform(test_df[['age_approx']]), columns=['scaled_age'])

## Dropping Original features.
train_df.drop(['sex', "anatom_site_general_challenge", 'age_approx'], axis=1, inplace=True)
test_df.drop(['sex', "anatom_site_general_challenge", 'age_approx'], axis=1, inplace=True)

## Concatenating the Original features.
train_df = pd.concat([train_df, train_sex_features, train_anatom_features, train_scaled_age], axis=1)
test_df = pd.concat([test_df, test_sex_features, test_anatom_features, test_scaled_age], axis=1)

## Defining Tabular-features
tabular_features = ['sex_missing', 'anatom_site_general_challenge_head_neck', 
                   'anatom_site_general_challenge_lower_extremity', 'anatom_site_general_challenge_torso', 
                   'anatom_site_general_challenge_upper_extremity', 'scaled_age']
num_tabular_features = len(tabular_features)

In [11]:
def create_folds(train_df=train_df):
    train_df = train_df.loc[train_df['tfrecord'] != -1].reset_index(drop=True)
    train_df['fold'] = train_df['tfrecord'] % 5
    return train_df

In [12]:
## Creating Image_Path for each images in 2019 & 2020 training datasets
train_df['image_path'] = os.path.join(Config.train_folder_2020) + train_df['image_name'] + ".jpg"
test_df['image_path'] = os.path.join(Config.test_folder_2020) + test_df['image_name'] + ".jpg"

train_df.head()

,image_name,patient_id,target,tfrecord,sex_male,sex_missing,anatom_site_general_challenge_head_neck,anatom_site_general_challenge_lower_extremity,anatom_site_general_challenge_torso,anatom_site_general_challenge_upper_extremity,scaled_age,image_path
0,ISIC_2637011,IP_7279968,0,0,1.0,0.0,1.0,0.0,0.0,0.0,-0.267090,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
1,ISIC_0015719,IP_3075186,0,0,0.0,0.0,0.0,0.0,0.0,1.0,-0.267090,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
2,ISIC_0052212,IP_2842074,0,6,0.0,0.0,0.0,1.0,0.0,0.0,0.080954,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
3,ISIC_0068279,IP_6890425,0,0,0.0,0.0,1.0,0.0,0.0,0.0,-0.267090,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
4,ISIC_0074268,IP_8723313,0,11,0.0,0.0,0.0,0.0,0.0,1.0,0.428998,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...


In [14]:
def run_model(fold, train_df):
    train_df = create_folds(train_df=train_df)
    train_data = train_df.loc[train_df['fold'] != fold].reset_index(drop=True)
    valid_data = train_df.loc[train_df['fold'] == fold].reset_index(drop=True)
    validation_targets = valid_data['target']
    train_dataset = DatasetRetriever(df=train_data, tabular_features=tabular_features, use_tabular_features=True, 
                     augmentations=training_augmentations, is_test=False)
    valid_dataset = DatasetRetriever(df=valid_data, tabular_features=tabular_features, use_tabular_features=True, 
                     augmentations=validation_augmentations, is_test=False)
    training_dataloader = DataLoader(dataset=train_dataset, batch_size=Config.TRAIN_BATCH_SIZE, 
                                     shuffle=True, num_workers=os.cpu_count())
    validation_dataloader = DataLoader(dataset=valid_dataset, batch_size=Config.VAL_BATCH_SIZE,
                                       shuffle=False, num_workers=os.cpu_count())
    seed_everything(Config.RANDOM_STATE)
    if torch.cuda.device_count() in (0,1):
        model = Model(num_tabular_features=num_tabular_features).to(Config.DEVICE)
    elif torch.cuda.device_count() > 1:
        model = Model(num_tabular_features=num_tabular_features).to(Config.DEVICE)
        model = nn.DataParallel(model)
    loss = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, 
                                    mode='max', factor=0.2, patience=2, 
                                    threshold=1e-3,verbose=True)
    scaler = amp.GradScaler()
    start_time = timer()
    model_save_path = f"Models/efficientnet_b5_checkpoint_fold_{fold}.pt"
    model_results = train(model=model, train_dataloader=training_dataloader, 
                            valid_dataloader=validation_dataloader,
                            loss_fn = loss, optimizer=optimizer, scheduler=scheduler,
                            device=Config.DEVICE, scaler=scaler,
                            epochs=Config.EPOCHS, es_patience=2, 
    model_save_path=model_save_path, validation_targets=validation_targets)
    end_time = timer()
    print(f"Total training time: {end_time-start_time:.3f} seconds")

In [23]:
def validate(fold, train_df, use_tabular_features=True):
    full_train_df = create_folds(train_df)
    valid_data = full_train_df.loc[full_train_df['fold'] == fold].reset_index(drop=True)
    validation_targets = valid_data['target']
    valid_dataset = DatasetRetriever(df=valid_data, tabular_features=tabular_features, use_tabular_features=True, 
                     augmentations=validation_augmentations, is_test=False)
    validation_dataloader = DataLoader(dataset=valid_dataset, batch_size=Config.VAL_BATCH_SIZE,
                                       shuffle=False, num_workers=os.cpu_count())
    valid_predictions = []
    if torch.cuda.device_count() in (0,1):
        model = Model(num_tabular_features=num_tabular_features).to(Config.DEVICE)
    elif torch.cuda.device_count() > 1:
        model = Model(num_tabular_features=num_tabular_features).to(Config.DEVICE)
        model = nn.DataParallel(model)
    model.load_state_dict(torch.load(f"/kaggle/working/Models/efficientnet_b5_checkpoint_fold_{fold}.pt"))
    model.eval()
    with torch.inference_mode():
        for batch, data in enumerate(validation_dataloader):
            if use_tabular_features:
                data["image"], data["tabular_features"], data['targets'] = data["image"].to(Config.DEVICE, dtype=torch.float), \
                    data["tabular_features"].to(Config.DEVICE, dtype=torch.float), data['targets'].to(Config.DEVICE, dtype=torch.float)
                y_logits = model(data['image'], data['tabular_features']).squeeze(dim=0)
            else:   
                data["image"], data['targets'] = data["image"].to(Config.DEVICE, dtype=torch.float),\
                    data['targets'].to(Config.DEVICE, dtype=torch.float)    
                y_logits = model(data['image']).squeeze(dim=0)
            valid_probs = torch.sigmoid(y_logits).detach().cpu().numpy()
            valid_predictions.extend(valid_probs)
    valid_auc = roc_auc_score(y_true=validation_targets, y_score=valid_predictions)
    print(valid_auc)

In [24]:
def predict_on_test(fold, use_tabular_features=True):
    test_dataset = DatasetRetriever(df=test_df, tabular_features=tabular_features, use_tabular_features=True, 
                     augmentations=testing_augmentations, is_test=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=Config.VAL_BATCH_SIZE,
                                       shuffle=False, num_workers=os.cpu_count())
    test_predictions = []
    if torch.cuda.device_count() in (0,1):
        model = Model(num_tabular_features=num_tabular_features).to(Config.DEVICE)
    elif torch.cuda.device_count() > 1:
        model = Model(num_tabular_features=num_tabular_features).to(Config.DEVICE)
        model = nn.DataParallel(model)
    model.load_state_dict(torch.load(f"/kaggle/working/Models/efficientnet_b5_checkpoint_fold_{fold}.pt"))   
    model.eval()
    with torch.inference_mode():
        for batch, data in enumerate(test_dataloader):
            if use_tabular_features:
                data["image"], data["tabular_features"] = data["image"].to(Config.DEVICE, dtype=torch.float), \
                        data["tabular_features"].to(Config.DEVICE, dtype=torch.float)
                y_logits = model(data['image'], data['tabular_features'])
            else:
                data["image"] = data["image"].to(Config.DEVICE, dtype=torch.float)    
                y_logits = model(data['image']).squeeze(dim=0)
            test_probs = torch.sigmoid(y_logits).detach().cpu().numpy()
            test_predictions.extend(test_probs)
    submission_df = pd.read_csv(Config.submission_csv_path)
    test_predictions = [test_predictions[img].item() for img in range(len(test_predictions))]
    submission_df['target'] = test_predictions
    submission_df.to_csv("submission.csv", index=False)

In [17]:
run_model(fold=0, train_df=train_df)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth


  0%|          | 0.00/117M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b5


  0%|          | 0/20 [00:00<?, ?it/s]

Using tabular features
Using tabular features
Validation loss decreased (inf --> 0.073976).  Saving model ...


...

Epoch : 3 | train_loss : 0.0690 | valid_loss : 0.0608 | valid_auc : 0.9183 
Using tabular features
Using tabular features
EarlyStopping counter: 1 out of 2


 20%|██        | 4/20 [3:29:46<13:59:15, 3147.24s/it]

Epoch : 4 | train_loss : 0.0670 | valid_loss : 0.0640 | valid_auc : 0.9032 
Using tabular features
Using tabular features


 20%|██        | 4/20 [4:22:02<17:28:09, 3930.60s/it]

EarlyStopping counter: 2 out of 2
Early Stopping
Total training time: 15722.408 seconds


In [25]:
validate(fold=0, train_df=train_df)

Loaded pretrained weights for efficientnet-b5
0.9182890775448209


In [26]:
predict_on_test(fold=0)

Loaded pretrained weights for efficientnet-b5


This model gave a score of 0.8828 on private leadeboard & 0.8942 on public leaderboard.